# Exploratory Data Analysis - Insurance Risk Analytics

This notebook contains exploratory data analysis for the insurance risk analytics project.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

# Matplotlib inline for Jupyter notebooks
%matplotlib inline

# Add src to path
sys.path.append(str(Path('..') / 'src'))

from utils.config import RAW_DATA_DIR, PROCESSED_DATA_DIR


## Load Data


In [ ]:
# Load raw data
data_path = RAW_DATA_DIR / 'insurance.csv'

if data_path.exists():
    df = pd.read_csv(data_path)
    
    # For this dataset, 'charges' represents insurance costs
    # We can treat it as claims for analysis purposes
    # If you have actual data with TransactionMonth, TotalClaims, TotalPremium, uncomment below:
    # if 'TransactionMonth' in df.columns:
    #     df['TransactionMonth'] = pd.to_datetime(df['TransactionMonth'])
    # if 'TotalClaims' in df.columns and 'TotalPremium' in df.columns:
    #     df['loss_ratio'] = df['TotalClaims'] / df['TotalPremium'].replace(0, np.nan)
    
    print(f"Data loaded: {df.shape}")
    print(df.head())
    print(f"\nColumns: {list(df.columns)}")
    print(f"\nData types:\n{df.dtypes}")
else:
    print(f"Data file not found at {data_path}")
    print("Please add insurance.csv to data/raw/ and track with DVC")


Data loaded: (1338, 7)
   age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520

Columns: ['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges']


## Data Overview


In [18]:
if 'df' in locals() and not df.empty:
    print("Dataset Info:")
    print(df.info())
    print("\nSummary Statistics:")
    print(df.describe())
    print("\nMissing Values:")
    print(df.isnull().sum())


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB
None

Summary Statistics:
               age          bmi     children       charges
count  1338.000000  1338.000000  1338.000000   1338.000000
mean     39.207025    30.663397     1.094918  13270.422265
std      14.049960     6.098187     1.205493  12110.011237
min      18.000000    15.960000     0.000000   1121.873900
25%      27.000000    26.296250     0.000000   4740.287150
50%      39.000000    30.400000     1.000000   9382.033000
75%      51.000000    34.69

## Visualizations

Create your EDA visualizations here. Aim for at least 3 creative plots that provide insights.


In [ ]:
# Creative Plot 2: Distribution of Charges with BMI Overlay
if 'df' in locals() and not df.empty:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Left: Distribution of charges
    ax1.hist(df['charges'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
    ax1.axvline(df['charges'].mean(), color='red', linestyle='--', linewidth=2, 
                label=f'Mean: ${df["charges"].mean():,.0f}')
    ax1.axvline(df['charges'].median(), color='green', linestyle='--', linewidth=2, 
                label=f'Median: ${df["charges"].median():,.0f}')
    ax1.set_xlabel('Charges ($)', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Frequency', fontsize=12, fontweight='bold')
    ax1.set_title('Distribution of Insurance Charges', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(alpha=0.3, linestyle='--')
    
    # Right: Charges by BMI categories
    df['BMI_Category'] = pd.cut(df['bmi'], 
                                bins=[0, 18.5, 25, 30, np.inf],
                                labels=['Underweight', 'Normal', 'Overweight', 'Obese'])
    bmi_charges = df.groupby('BMI_Category')['charges'].mean().sort_values()
    
    colors_bmi = plt.cm.RdYlGn_r(np.linspace(0.3, 0.9, len(bmi_charges)))
    bars = ax2.bar(range(len(bmi_charges)), bmi_charges.values, color=colors_bmi, 
                    edgecolor='black', alpha=0.8)
    ax2.set_xlabel('BMI Category', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Average Charges ($)', fontsize=12, fontweight='bold')
    ax2.set_title('Average Charges by BMI Category', fontsize=14, fontweight='bold')
    ax2.set_xticks(range(len(bmi_charges)))
    ax2.set_xticklabels(bmi_charges.index, rotation=45, ha='right')
    ax2.grid(axis='y', alpha=0.3, linestyle='--')
    
    # Add value labels on bars
    for i, (idx, val) in enumerate(bmi_charges.items()):
        ax2.text(i, val, f'${val:,.0f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("Average charges by BMI category:")
    print(bmi_charges)


In [ ]:
# Creative Plot 3: Heatmap - Charges by Age Group, Children, and Smoking Status
if 'df' in locals() and not df.empty:
    # Create age groups
    df['AgeGroup'] = pd.cut(df['age'], 
                            bins=[0, 30, 40, 50, 60, 100],
                            labels=['18-30', '31-40', '41-50', '51-60', '60+'])
    
    # Create pivot table: Age Group vs Children, colored by average charges
    pivot_data = df.groupby(['AgeGroup', 'children', 'smoker'])['charges'].mean().reset_index()
    
    # Create heatmap for smokers
    pivot_smoker = pivot_data[pivot_data['smoker'] == 'yes'].pivot(
        index='AgeGroup', columns='children', values='charges'
    )
    
    # Create heatmap for non-smokers
    pivot_non_smoker = pivot_data[pivot_data['smoker'] == 'no'].pivot(
        index='AgeGroup', columns='children', values='charges'
    )
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))
    
    # Smokers heatmap
    sns.heatmap(pivot_smoker, annot=True, fmt='.0f', cmap='Reds', ax=ax1,
                cbar_kws={'label': 'Average Charges ($)'}, linewidths=0.5, linecolor='gray')
    ax1.set_title('Average Charges: Smokers\n(Age Group × Number of Children)', 
                  fontsize=14, fontweight='bold', pad=15)
    ax1.set_xlabel('Number of Children', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Age Group', fontsize=12, fontweight='bold')
    
    # Non-smokers heatmap
    sns.heatmap(pivot_non_smoker, annot=True, fmt='.0f', cmap='Blues', ax=ax2,
                cbar_kws={'label': 'Average Charges ($)'}, linewidths=0.5, linecolor='gray')
    ax2.set_title('Average Charges: Non-Smokers\n(Age Group × Number of Children)', 
                  fontsize=14, fontweight='bold', pad=15)
    ax2.set_xlabel('Number of Children', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Age Group', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("Heatmap data summary:")
    print(f"\nSmokers - Min: ${pivot_smoker.min().min():,.0f}, Max: ${pivot_smoker.max().max():,.0f}")
    print(f"Non-Smokers - Min: ${pivot_non_smoker.min().min():,.0f}, Max: ${pivot_non_smoker.max().max():,.0f}")


In [ ]:
# Creative Plot 1: Average Charges by Region (with smoker overlay)
if 'df' in locals() and not df.empty:
    plt.figure(figsize=(14, 8))
    
    # Group by region and smoker status
    region_smoker = df.groupby(['region', 'smoker'])['charges'].mean().unstack()
    
    x = np.arange(len(region_smoker.index))
    width = 0.35
    
    bars1 = plt.bar(x - width/2, region_smoker['no'], width, label='Non-Smoker', 
                    color='steelblue', alpha=0.8)
    bars2 = plt.bar(x + width/2, region_smoker['yes'], width, label='Smoker', 
                    color='coral', alpha=0.8)
    
    plt.xlabel('Region', fontsize=12, fontweight='bold')
    plt.ylabel('Average Charges ($)', fontsize=12, fontweight='bold')
    plt.title('Average Insurance Charges by Region and Smoking Status', 
              fontsize=14, fontweight='bold', pad=20)
    plt.xticks(x, region_smoker.index, rotation=45, ha='right')
    plt.legend(fontsize=11)
    plt.grid(axis='y', alpha=0.3, linestyle='--')
    plt.tight_layout()
    plt.show()
    
    print("Average charges by region and smoking status:")
    print(region_smoker)
    print(f"\nSmoker premium: {region_smoker['yes'].mean():.2f}")
    print(f"Non-smoker premium: {region_smoker['no'].mean():.2f}")
    print(f"Smoker premium is {region_smoker['yes'].mean() / region_smoker['no'].mean():.2f}x higher")


## Hypothesis Testing

Perform hypothesis tests to identify low-risk groups.


In [12]:
from scipy import stats

# Hypothesis test: Loss ratio by Province
if 'df' in locals() and not df.empty:
    if 'Province' in df.columns and 'loss_ratio' in df.columns:
        prov = df.groupby('Province').agg({'TotalClaims':'sum','TotalPremium':'sum'})
        prov['loss_ratio'] = prov['TotalClaims']/prov['TotalPremium']
        prov_sorted = prov.sort_values('loss_ratio')
        
        if len(prov_sorted) >= 2:
            lowest_prov = prov_sorted.index[0]
            highest_prov = prov_sorted.index[-1]
            
            lowest_data = df[df['Province'] == lowest_prov]['loss_ratio'].dropna()
            highest_data = df[df['Province'] == highest_prov]['loss_ratio'].dropna()
            
            if len(lowest_data) > 0 and len(highest_data) > 0:
                t_stat, p_value = stats.ttest_ind(lowest_data, highest_data)
                print(f"T-test: {lowest_prov} (lowest) vs {highest_prov} (highest) loss ratio")
                print(f"T-statistic: {t_stat:.4f}")
                print(f"P-value: {p_value:.4f}")
                print(f"Significant: {p_value < 0.05}")
                print(f"\n{lowest_prov} mean loss ratio: {lowest_data.mean():.4f}")
                print(f"{highest_prov} mean loss ratio: {highest_data.mean():.4f}")


## Summary

Document your findings and insights here.
